In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import isaacgym
import isaacgymenvs
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator

from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder

from omegaconf import DictConfig, OmegaConf

import torch
import numpy as np
import matplotlib.pyplot as plt

from isaacgymenvs.qp.fista import ForceQP, LocationQP, FISTA
from isaacgymenvs.qp.qp_utils import *
from isaacgymenvs.qp.vecrobotics import *

Importing module 'gym_38' (/home/hz/devel/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/hz/devel/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json


PyTorch version 2.0.0
Device count 1
/home/hz/devel/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/hz/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/hz/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...
2023-06-21 16:38:33,851 - INFO - logger - logger initialized
<unknown>:3: DeprecationWarning: invalid escape sequence \*
/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


In [3]:
cfg = OmegaConf.load("../isaacgymenvs/cfg/config.yaml")
cfg.task_name = "TrifingerNYU"
cfg.num_envs = 4
cfg.task = OmegaConf.load("../isaacgymenvs/cfg/task/TrifingerNYU.yaml")

cfg.task.env.command_mode = "torque"
cfg.task.env.enable_location_qp = False
cfg.task.env.enable_force_qp = False

cfg.task.env.reset_distribution.object_initial_state.type = "default"
cfg.task.env.reset_distribution.robot_initial_state.type = "default"

cfg.task.normalize_action = False
cfg.task.env.enable_ftip_damping = False
cfg.headless = False

In [4]:
def create_env_thunk(**kwargs):
    envs = isaacgymenvs.make(
            cfg.seed, 
            cfg.task_name, 
            cfg.task.env.numEnvs, 
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
    return envs

In [5]:
lifting_data = np.load("data/lifting2.npz", allow_pickle=True)["data"]

In [6]:
device = cfg.sim_device
envs = create_env_thunk()

/home/hz/miniforge3/envs/rlgpu/lib/python3.8/site-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/hz/learning/IsaacGymEnvs/isaacgymenvs/tasks/trifingerNYU.py:1424: DeprecationWarning: an integer is required (got type isaacgym._bindings.linux-x86_64.gym_38.DofDriveMode).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  robot_asset_options.default_dof_drive_mode = gymapi.DOF_MODE_EFFORT


[Warning] [carb.gym.plugin] useGpu is set, forcing single scene (0 subscenes)
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Trifinger Robot Asset: 
	 Number of bodies: 23
	 Number of shapes: 39
	 Number of dofs: 9
	 Number of actuated dofs: 9
Trifinger Table Asset: 
	 Number of bodies: 2
	 Number of shapes: 1
Trifinger Boundary Asset: 
	 Number of bodies: 1
	 Number of shapes: 41
Using VHACD cache directory '/home/hz/.isaacgym/vhacd'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/int_sim.stl'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/prox-sim.stl'
Found existing convex decomposition for mesh '/home/hz/learning/IsaacGymEnvs/isaacgymenvs/../assets/trifinger/robot_properties_fingers/meshes/pro/tip_link_sim.stl'
Found existing co

In [7]:
def get_force_qp_data(ftip_pos: torch.Tensor, object_pose: torch.Tensor, 
                      total_force_des_object_frame: torch.Tensor, 
                      total_torque_des_object_frame: torch.Tensor,
                      torque_ref: torch.Tensor, jacobian: torch.Tensor,
                      weights: List[float]):
    # get ftip positin in the object frame
    batch_size, num_ftip, _ = ftip_pos.shape
    num_vars = num_ftip * 3

    p = SE3_inverse_transform(object_pose.repeat_interleave(3, dim=0), ftip_pos.view(-1, 3))
    contact_normals = get_cube_contact_normals(p)
    object_orn = quat2mat(object_pose[:, 3:])
    jacobian_transpose = torch.transpose(jacobian, 1, 2)
    
    
    # force cost
    R = get_contact_frame_orn(contact_normals)
    R_vstacked = R.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
    Q1 = R_vstacked @ R_vstacked.transpose(1, 2)
    q1 = -2 * bmv(R_vstacked, total_force_des_object_frame)
    
    # torque cost
    pxR = vec2skewsym_mat(p) @ R
    pxR_vstacked = pxR.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
    Q2 = pxR_vstacked @ pxR_vstacked.transpose(1, 2)
    q2 = -2 * bmv(pxR_vstacked, total_torque_des_object_frame)
    
    # joint torque cost
    R_reshaped = R.view(batch_size, num_ftip, 3, 3)
    R_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
    for i in range(num_ftip):
        R_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = R_reshaped[:, i]
    object_orn_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
    for i in range(num_ftip):
        object_orn_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = object_orn
    A = jacobian_transpose @ object_orn_diag @ R_diag
    Q3 = torch.transpose(A, 1, 2) @ A
    q3 = -2 * bmv(A, torque_ref)
    
    # regularization
    Q4 = 1e-4 * torch.eye(3 * num_ftip).repeat(batch_size, 1, 1).to(ftip_pos.device)
    
    # If any of contact normals == 0, set the desired force and torque weights to zero
    reshaped_tensor = Q1.view(batch_size, -1)
    mask = torch.any(reshaped_tensor[:, ::num_vars+1] == 0, dim=1)

    # construct total cost
    weights = torch.tensor(weights, dtype=torch.float32, device=ftip_pos.device).repeat(batch_size, 1)
    w1, w2, w3 = weights.split(1, dim=1)
    w1[mask] = 0
    w2[mask] = 0
    w3[mask] = 0
    
    q = w1*q1 + w2*q2 + w3*q3
    
    w1 = w1.view(batch_size, 1, 1)
    w2 = w2.view(batch_size, 1, 1)
    w3 = w3.view(batch_size, 1, 1)
    
    Q = w1 * Q1 + w2 * Q2 + w3 * Q3 + Q4
    
    return Q, q, R_vstacked, pxR_vstacked, contact_normals

In [8]:
# get ftip states
envs.reset_idx(torch.arange(cfg.num_envs))
N = 700
action_buffer = torch.zeros(N, 9).to(device)
projected_torque_buffer = torch.zeros(N, 9).to(device)

ftip_pos_buffer = torch.zeros(N, 3, 3).to(device)
ftip_pos_local_buffer = torch.zeros(N, 3, 3).to(device)
object_pose_buffer = torch.zeros(N, 7).to(device)
contact_normals_buffer = []

q_buffer = torch.zeros(N, 9).to(device)
dq_buffer = torch.zeros(N, 9).to(device)
obs, rwds, resets, info = envs.step(torch.zeros(cfg.num_envs, 9))

In [9]:
# construct force QP
batch_size = cfg.num_envs
num_vars = 9
force_lb = -1.5 * torch.ones(batch_size, num_vars)
force_ub = 1.5 * torch.ones(batch_size, num_vars)

force_qp = ForceQP(batch_size, num_vars, friction_coeff=1.0, device=device)
force_qp_solver = FISTA(force_qp, device=device)
max_it = 20

for n in range(N):
    q = envs._dof_position
    dq = envs._dof_velocity
    q_buffer[n] = q[0]
    dq_buffer[n] = dq[0]
    
    ftip_state = envs._rigid_body_state[:, envs._fingertip_indices]
    ftip_pos = ftip_state[:, :, 0:3]
    ftip_vel = ftip_state[:, :, 7:10]
    ftip_pos_buffer[n] = ftip_pos[0]
    
    object_pose = envs._object_state_history[0][:, 0:7]
    object_position = object_pose[:, 0:3]
    desired_object_position = envs._desired_object_poses_buf[:, 0:3]
    object_orn = quat2mat(object_pose[:, 3:])
    object_pose_buffer[n] = object_pose[0]
    
    time_idx = 0 if n < 200 else 20 * (n - 200)
    ftip_pos_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_pos_des'], dtype=torch.float32).to(device)
    ftip_vel_des = torch.tensor(lifting_data[time_idx]['policy']['controller']['ft_vel_des'], dtype=torch.float32).to(device)

    ftip_pos_diff = ftip_pos_des - ftip_pos.reshape(cfg.num_envs, 9)
    ftip_vel_diff = ftip_vel_des - ftip_vel.reshape(cfg.num_envs, 9)
    
    task_space_force = torch.tensor([200,200,200]*3, dtype=torch.float32, device=envs.device) * ftip_pos_diff
    task_space_force += torch.tensor([5]*9, dtype=torch.float32, device=envs.device)  * ftip_vel_diff
    
    jacobian = envs.get_fingertip_jacobian_linear()
    jacobian_transpose = torch.transpose(jacobian, 1, 2)
    
#     action = -0.36 + 0.72 * torch.rand(cfg.num_envs, 9).to(device)
#     task_space_force = -1.5 + 3. * torch.rand(cfg.num_envs, 9).to(device)
    
    torque_ref = bmv(jacobian_transpose, task_space_force)
    
    desired_object_acceleration = 10 * (desired_object_position - object_position)
    desired_object_acceleration += torch.tensor([0, 0, 9.81]).to(desired_object_acceleration.device)
    total_force_des = 0.08 * desired_object_acceleration
    total_force_des_object_frame = bmv(object_orn.transpose(1,2), total_force_des)
    total_torque_des_object_frame = torch.zeros_like(total_force_des_object_frame)
    force_qp_cost_weights = [1, 200, 0.01]
    Q, q, R_vstacked, pxR_vstacked, contact_normals = get_force_qp_data(ftip_pos, 
                                                                        object_pose,
                                                                        total_force_des_object_frame,
                                                                        total_torque_des_object_frame,
                                                                        torque_ref, 
                                                                        jacobian,
                                                                        force_qp_cost_weights)
    
    force_qp_solver.prob.set_data(Q, q, force_lb, force_ub)
    force_qp_solver.reset()
    for i in range(max_it):
        force_qp_solver.step()
    ftip_force_contact_frame = force_qp_solver.prob.yk.clone()

    # convert force to the world frame
    R = R_vstacked.reshape(-1, 3, 3).transpose(1, 2)
    ftip_force_object_frame = stacked_bmv(R, ftip_force_contact_frame)
    object_orn = quat2mat(object_pose[:, 3:7]).repeat(3, 1, 1)
    task_space_force2 = stacked_bmv(object_orn, ftip_force_object_frame)
    
    action = torque_ref + bmv(jacobian_transpose, task_space_force2)
    
    action_buffer[n] = action[0]
#     projected_torque_buffer[n] = projected_torque[0]
    obs, rwds, resets, info = envs.step(action)

tensor([3., 3., 3., 3.], device='cuda:0')

In [10]:
# weights = force_qp_cost_weights
# # get ftip positin in the object frame
# batch_size, num_ftip, _ = ftip_pos.shape
# num_vars = num_ftip * 3

# p = SE3_inverse_transform(object_pose.repeat_interleave(3, dim=0), ftip_pos.view(-1, 3))
# contact_normals = get_cube_contact_normals(p)
# object_orn = quat2mat(object_pose[:, 3:])
# jacobian_transpose = torch.transpose(jacobian, 1, 2)


# # force cost
# R = get_contact_frame_orn(contact_normals)
# R_vstacked = R.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
# Q1 = R_vstacked @ R_vstacked.transpose(1, 2)
# total_force_des_object_frame = bmv(object_orn.transpose(1,2), total_force_des)
# q1 = -2 * bmv(R_vstacked, total_force_des_object_frame)

# # torque cost
# pxR = vec2skewsym_mat(p) @ R
# pxR_vstacked = pxR.transpose(1, 2).reshape(-1, 3 * num_ftip, 3)
# Q2 = pxR_vstacked @ pxR_vstacked.transpose(1, 2)

# # joint torque cost
# R_reshaped = R.view(batch_size, num_ftip, 3, 3)
# R_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
# for i in range(num_ftip):
#     R_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = R_reshaped[:, i]
# object_orn_diag = torch.zeros(batch_size, 9, 9).to(ftip_pos.device)
# for i in range(num_ftip):
#     object_orn_diag[:, i*3:i*3 + 3, i*3:i*3 + 3] = object_orn
# A = jacobian_transpose @ object_orn_diag @ R_diag
# Q3 = torch.transpose(A, 1, 2) @ A
# q3 = -2 * bmv(A, action)

# # regularization
# Q4 = 1e-4 * torch.eye(3 * num_ftip).repeat(batch_size, 1, 1).to(ftip_pos.device)

# # If any of contact normals == 0, set the desired force and torque weights to zero
# reshaped_tensor = Q1.view(batch_size, -1)
# mask = torch.any(reshaped_tensor[:, ::num_vars+1] == 0, dim=1)

# # construct total cost
# weights = torch.tensor(weights, dtype=torch.float32, device=ftip_pos.device).repeat(batch_size, 1)
# w1, w2, w3 = weights.split(1, dim=1)
# w1[mask] = 0
# w2[mask] = 0
# w3[mask] = 0

# q = w1*q1 + w3*q3

# w1 = w1.view(batch_size, 1, 1)
# w2 = w2.view(batch_size, 1, 1)
# w3 = w3.view(batch_size, 1, 1)

# Q = w1 * Q1 + w2 * Q2 + w3 * Q3 + Q4

In [11]:
task_space_force[:, :3] + task_space_force[:, 3:6] + task_space_force[:, 6:]

tensor([[-0.3937, -0.2288,  9.3105],
        [-0.4928, -0.3676,  9.3019],
        [-0.5745, -0.4299,  9.2759],
        [-0.3807, -0.2426,  9.3050]], device='cuda:0')

In [12]:
task_space_force2[:, :3] + task_space_force2[:, 3:6] + task_space_force2[:, 6:]

tensor([[-0.1205, -0.0657,  0.7112],
        [ 0.0035,  0.0066,  0.6878],
        [ 0.0665,  0.0289,  0.7119],
        [-0.1271, -0.0630,  0.7244]], device='cuda:0')

In [13]:
total_force_des

tensor([[-0.0492, -0.0549,  0.7845],
        [ 0.0464,  0.0361,  0.7794],
        [ 0.0433,  0.0532,  0.7975],
        [-0.1063, -0.0524,  0.7839]], device='cuda:0')

In [14]:
torque_ref

tensor([[-0.0805, -0.5506, -0.7373,  0.3386, -0.0197, -0.2880, -0.3727,  0.2157,
         -0.2410],
        [-0.1559, -0.5502, -0.7640,  0.3458, -0.0636, -0.2902, -0.3443,  0.2869,
         -0.2071],
        [-0.1101, -0.5419, -0.7533,  0.3709, -0.0759, -0.3034, -0.2858,  0.2980,
         -0.1760],
        [-0.0188, -0.5462, -0.7373,  0.3835, -0.0105, -0.2760, -0.3129,  0.2116,
         -0.2191]], device='cuda:0')

In [15]:
action

tensor([[-0.0828, -0.6934, -0.8773,  0.4321, -0.1051, -0.3802, -0.3754,  0.2543,
         -0.1653],
        [-0.1534, -0.7002, -0.9094,  0.4586, -0.1210, -0.3641, -0.3649,  0.3162,
         -0.1376],
        [-0.1161, -0.6976, -0.9057,  0.4940, -0.1327, -0.3775, -0.3067,  0.3266,
         -0.1051],
        [-0.0242, -0.6914, -0.8791,  0.4750, -0.0969, -0.3687, -0.3188,  0.2494,
         -0.1454]], device='cuda:0')

In [16]:
task_space_force2

tensor([[ 0.0159, -0.6414,  0.6413, -0.0202,  0.6052,  0.6027, -0.1163, -0.0295,
         -0.5328],
        [-0.0009, -0.6707,  0.6704,  0.1218,  0.6139,  0.5598, -0.1174,  0.0635,
         -0.5425],
        [ 0.0334, -0.6950,  0.6959,  0.1486,  0.6536,  0.5727, -0.1155,  0.0703,
         -0.5567],
        [ 0.0242, -0.6486,  0.6455, -0.0270,  0.6006,  0.6007, -0.1242, -0.0150,
         -0.5219]], device='cuda:0')

In [17]:
zero_rows = (task_space_force2.sum(dim=1) == 0)  # find rows in M1 that are all zeros
task_space_force2[zero_rows] = task_space_force[zero_rows]  

In [18]:
task_space_force2

tensor([[ 0.0159, -0.6414,  0.6413, -0.0202,  0.6052,  0.6027, -0.1163, -0.0295,
         -0.5328],
        [-0.0009, -0.6707,  0.6704,  0.1218,  0.6139,  0.5598, -0.1174,  0.0635,
         -0.5425],
        [ 0.0334, -0.6950,  0.6959,  0.1486,  0.6536,  0.5727, -0.1155,  0.0703,
         -0.5567],
        [ 0.0242, -0.6486,  0.6455, -0.0270,  0.6006,  0.6007, -0.1242, -0.0150,
         -0.5219]], device='cuda:0')